## Notebook Purpose

Prototype using probabilistic programming in NumPyro for control synthesis with STL specifications.

In [30]:
import torch

import numpyro
import numpyro.distributions as dist
from numpyro.infer import MCMC, NUTS

import numpy as np
import jax.numpy as jnp
from jax import random

In [47]:
# Let's define a simple model for a LTI dynamical system. No STL yet, but we just want to measure
# the distance between the output and setpoint after 5 seconds.
def lti_stability_model(damping=None, actuation=None, final_location=None):
    # The drift and actuation matrices A and B are random, but
    # are centered around a damped single integrator
    damping = numpyro.sample("damping", dist.Uniform(np.array(0.05), np.array(0.15)), obs=damping)
    actuation = numpyro.sample("actuation", dist.Uniform(np.array(0.8), np.array(1.2)), obs=actuation)

    A = np.zeros((damping.shape[0], 2, 2))
    A[:, 0, 1] = 1.0
    A[:, 1, 1] = -damping
    B = np.zeros((damping.shape[0], 2, 1))
    B[:, 1, 0] = actuation

    # Define a Gausian prior over the control parameters
    K = numpyro.sample("K", dist.Normal(np.ones((2, 1)), 5.0 * np.ones(2)))

    print(A.shape)
    print(B.shape)
    print(K.shape)

    # Define the closed-loop transfer matrix
    Acl = A - np.matmul(B, K)

    # Simulate for 5 seconds
    dt = 0.1
    T = 5.0
    n_steps = T // dt
    x = np.zeros((n_steps, 2, 1))
    x[0, 0, 1] = 1.0  # start at location = 1.0, velocity = 0.0
    for i in range(1, n_steps):
        x[i] = Acl @ x[i - 1]

    # Observe the final location
    observation_noise = 0.01
    numpyro.sample("observed_location", dist.Normal(x[-1, 0, 0], observation_noise), obs=final_location)

In [48]:
# Get an RNG key to work with
rng_key = random.PRNGKey(0)
rng_key, rng_subkey = random.split(rng_key)

# Get some damping values
N_samples = 10
damping = random.uniform(rng_subkey, (N_samples,), minval=0.05, maxval=0.15)
rng_key, rng_subkey = random.split(rng_key)
actuation = random.uniform(rng_subkey, (N_samples,), minval=0.8, maxval=1.2)
rng_key, rng_subkey = random.split(rng_key)
final_location = 0.01 * random.normal(rng_subkey, (N_samples,))

# Run NUTS.
kernel = NUTS(lti_stability_model)
num_samples = 2000
mcmc = MCMC(kernel, num_warmup=1000, num_samples=num_samples)
mcmc.run(
    rng_subkey, damping=damping, actuation=actuation, final_location=final_location,
)
mcmc.print_summary()
samples_1 = mcmc.get_samples()

(10, 2, 2)
(10, 2, 1)
(2, 2)


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 2 is different from 1)